1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
- 토큰수 초과로 답변을 생성하지 못할 수 있고
- 문서가 길면 (인풋이 길면) 답변 생성이 오래거림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [3]:
%pip install --upgrade --quiet docx2txt langchain-community
%pip install -qU langchain-text-splitters

You should consider upgrading via the '/Users/heewongwak/.pyenv/versions/inflearn-llm-application/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/heewongwak/.pyenv/versions/inflearn-llm-application/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 392 kB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 427 kB 11.3 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 11.3 MB/s eta 0:00:01
     |████████████████████████████████| 343 kB 11.0 MB/s eta 0:00:01
     |████████████████████████████████| 87 kB 9.5 MB/s  eta 0:00:01
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.11
    Uninstalling aiohttp-3.11.11:
      Successfully uninstalled aiohttp-3.11.11
You should consider upgrading via the '/Users/heewongwak/.pyenv/vers

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1500,
  chunk_overlap=200, # 문서를 겹치게 하면 유사도 검색을 할 때 우리가 원하는 문서를 가지고 올 확률을 높임
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter)


/Users/heewongwak/.pyenv/versions/inflearn-llm-application/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv() # OpenAIEmbeddings의 로드를 위해서 필요

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
%pip install langchain-pinecone

In [6]:
import os

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pinecone_api_key

from pinecone import Pinecone

pc = Pinecone(api_key=pinecone_api_key)

In [9]:

from langchain_pinecone import PineconeVectorStore

database = PineconeVectorStore.from_documents(
  documents=document_list,
  embedding=embeddings,
  index_name="tax-index"
)

In [22]:
query = "연봉 5억인 직장인의 소득세는 얼마인가요?" # 존댓말로 하면 답변이 더 좋을 수 있다? 라는 썰이 있다고 한다.
retrieved_docs = database.similarity_search(query, k=3) # 내 쿼리에 맞춰 유사도 검색을 하는데 관련 문서(데이터)를 3개를 가져온다.

In [17]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [18]:
%pip install -U langchain langchainhub --quiet

You should consider upgrading via the '/Users/heewongwak/.pyenv/versions/inflearn-llm-application/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [19]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

/Users/heewongwak/.pyenv/versions/inflearn-llm-application/lib/python3.9/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [20]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
  llm=llm,
  retriever=database.as_retriever(),
  chain_type_kwargs={"prompt": prompt}
)

In [23]:
ai_message = qa_chain({"query": query})
ai_message

{'query': '연봉 5억인 직장인의 소득세는 얼마인가요?',
 'result': '죄송하지만, 제공된 문맥에서는 연봉 5억 원인 직장인의 소득세를 계산하는 데 필요한 구체적인 세율이나 공제 항목에 대한 정보가 없습니다. 소득세 계산을 위해서는 해당 연도의 세율표와 공제 항목을 참조해야 합니다. 한국의 국세청 웹사이트나 세무 전문가에게 문의하는 것이 정확한 정보를 얻는 데 도움이 될 것입니다.'}